In [1]:
# Import the required modules
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import pickle as pkl
import random

# Pre-Processing
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

# Models
from sklearn.neighbors import NearestNeighbors

# suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("anime_ml.csv")
print(df.shape)
df.head()

(11903, 49)


,anime_id,name,episodes,rating,members,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Fantasy,Game,Historical,Horror,Josei,Kids,Magic,MartialArts,Mecha,Military,Music,Mystery,Parody,Police,Psychological,Romance,Samurai,School,Sci-Fi,Seinen,Shoujo,ShoujoAi,Shounen,ShounenAi,SliceofLife,Space,Sports,SuperPower,Supernatural,Thriller,Vampire,Movie,Music.1,ONA,OVA,Special,TV
0,32281,Kimi no Na wa.,1,9.37,200630,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
1,5114,Fullmetal Alchemist: Brotherhood,64,9.26,793665,1,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,28977,Gintama,51,9.25,114262,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,9253,Steins Gate,24,9.17,673572,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
4,9969,Gintama',51,9.16,151266,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [3]:
df.describe()

,anime_id,episodes,rating,members,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Fantasy,Game,Historical,Horror,Josei,Kids,Magic,MartialArts,Mecha,Military,Music,Mystery,Parody,Police,Psychological,Romance,Samurai,School,Sci-Fi,Seinen,Shoujo,ShoujoAi,Shounen,ShounenAi,SliceofLife,Space,Sports,SuperPower,Supernatural,Thriller,Vampire,Movie,Music.1,ONA,OVA,Special,TV
count,11903.000000,11903.000000,11903.000000,1.190300e+04,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000
mean,13515.515080,12.426279,6.484570,1.844488e+04,0.233555,0.193901,0.005965,0.378728,0.019995,0.024364,0.166765,0.189028,0.014618,0.066454,0.030497,0.004369,0.130975,0.063849,0.021675,0.078216,0.035033,0.070234,0.040830,0.033437,0.016382,0.019071,0.121314,0.012182,0.098715,0.171049,0.044695,0.049567,0.004537,0.141057,0.005377,0.099555,0.031589,0.044611,0.037554,0.084601,0.007225,0.008485,0.193397,0.040914,0.051164,0.273881,0.140217,0.300428
std,11174.446066,46.959892,1.016017,5.521622e+04,0.423110,0.395369,0.077005,0.485091,0.139989,0.154182,0.372781,0.391547,0.120024,0.249084,0.171956,0.065954,0.337388,0.244495,0.145627,0.268522,0.183871,0.255552,0.197905,0.179782,0.126946,0.136780,0.326505,0.109701,0.298291,0.376568,0.206641,0.217058,0.067205,0.348095,0.073132,0.299418,0.174910,0.206456,0.190122,0.278298,0.084696,0.091728,0.394978,0.198099,0.220341,0.445967,0.347226,0.458464
min,1.000000,1.000000,1.670000,5.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3361.500000,1.000000,5.900000,2.340000e+02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,9885.000000,2.000000,6.570000,1.596000e+03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,23485.000000,12.000000,7.180000,9.830000e+03,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000
max,34527.000000,1818.000000,10.000000,1.013917e+06,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000

In [26]:
def make_recommendation(anime_length, anime_name):
    # Step 1: Load the dataset
    df = pd.read_csv("anime_ml.csv")
    
    # Remove any rows with missing values and reset the index
    df = df.dropna(how="any").reset_index(drop=True)

    # # Extract the release year from the album release date and convert to integer
    # df["track_release_year"] = df.track_album_release_date.apply(lambda x: int(x[0:4]))

    # # Drop unnecessary columns that are not needed for recommendation
    # df = df.drop(columns=["track_album_id", "track_album_name", "track_album_release_date", 
    #                       "playlist_name", "playlist_id"])
    # Update the Column name
    df.rename(columns={'Music.1': 'Music'}, inplace=True)
    
    # Remove duplicate anime based on track_id and reset index again
    df = df.drop_duplicates(subset=["name"]).reset_index(drop=True)

    # Step 2: Define the columns for metadata and features
    meta_cols = ["anime_id","name"]  # Metadata columns to keep anime info
    feature_cols = ['episodes', 'rating', 'members', 'Action', 'Adventure', 'Cars',
       'Comedy', 'Dementia', 'Demons', 'Drama', 'Fantasy', 'Game',
       'Historical', 'Horror', 'Josei', 'Kids', 'Magic', 'MartialArts',
       'Mecha', 'Military', 'Music', 'Mystery', 'Parody', 'Police',
       'Psychological', 'Romance', 'Samurai', 'School', 'Sci-Fi', 'Seinen',
       'Shoujo', 'ShoujoAi', 'Shounen', 'ShounenAi', 'SliceofLife', 'Space',
       'Sports', 'SuperPower', 'Supernatural', 'Thriller', 'Vampire', 'Movie',
       'Music', 'ONA', 'OVA', 'Special', 'TV']

    # Step 3: Define preprocessing steps for the data

    # Preprocessing for numeric features: fill missing values with the mean and standardize the values
    numeric_features = []
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),  # Fill missing values with the mean
        ('scaler', StandardScaler())])  # Standardize features

    # Preprocessing for binary features (if any): fill missing values with most frequent and label encode
    binary_features = []
    binary_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent', missing_values=pd.NA)),  # Fill missing values
        ('label', OrdinalEncoder())])  # Convert binary features to ordinal values

    # Preprocessing for categorical features: fill missing values and apply one-hot encoding
    categorical_features = []
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent', missing_values=pd.NA)),  # Fill missing values
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])  # Apply one-hot encoding

    # Combine all preprocessing steps into a single preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),  # Numeric features preprocessing
            ('binary', binary_transformer, binary_features),  # Binary features preprocessing (if any)
            ('cat', categorical_transformer, categorical_features)])  # Categorical features preprocessing

    # Step 4: Get the target track's ID based on track name and artist
    # Select the most popular track by track_name and track_artist combination
    anime_id = df.loc[(df.name == anime_name)] \
                .sort_values(by="rating", ascending=False).anime_id.values[0]
    # anime_id = 22319
    # Step 5: Prepare the feature matrix for the Nearest Neighbors model
    X = df.loc[:, feature_cols]  # Select the feature columns

    # Apply preprocessing to the feature matrix
    # preprocessor.fit(X)  # Fit the preprocessor to the feature matrix
    # X_preprocessed = preprocessor.transform(X)  # Transform the feature matrix

    # Step 6: Initialize the Nearest Neighbors model
    # # define the number of nearest neighbors to consider and use cosine similarity as the metric
    k = anime_length
    model1 = NearestNeighbors(n_neighbors=k, metric="cosine")

    # Fit the Nearest Neighbors model to the preprocessed data
    # model1.fit(X_preprocessed)
    model1.fit(X)
    # Step 7: Extract features of the anime
    anime_features = df.loc[df.anime_id == anime_id, feature_cols]  # Get the feature vector for the target anime
    # anime_features_preprocessed = preprocessor.transform(anime_features)  # Preprocess the target track features

    # Step 8: Find the nearest neighbors (anime most similar to the target selection)
    # distances, indices = model1.kneighbors(anime_features_preprocessed)  # Get distances and indices of neighbors
    distances, indices = model1.kneighbors(anime_features)  # Get distances and indices of neighbors

    # Step 9: Retrieve the metadata of the recommended tracks
    animes = df.iloc[indices[0]]  # Select anime corresponding to the nearest neighbors
    animes["distance"] = distances[0]  # Add the distance of each neighbor as a new column

    # Step 10: Filter the columns for the final output
    cols = animes.columns # you can explicitly choose to return specific columns here
    animes = animes.loc[:, cols]  # Keep the relevant columns
    animes = animes.sort_values(by="distance")  # Sort the tracks by their distance (most similar first)

    # Step 11: Return the recommended tracks as a list of dictionaries
    return animes.to_dict(orient="records")

In [ ]:
 # User input
anime_length = 10
anime_name = "Naruto"


response = make_recommendation(anime_length, anime_name) # used for the flask app, returns JSON

# to test
pd.DataFrame(response)

In [6]:
def make_recommendation_euc(anime_length, anime_name):
    # Step 1: Load the dataset
    df = pd.read_csv("anime_ml.csv")
    
    # Remove any rows with missing values and reset the index
    df = df.dropna(how="any").reset_index(drop=True)

    # # Extract the release year from the album release date and convert to integer
    # df["track_release_year"] = df.track_album_release_date.apply(lambda x: int(x[0:4]))

    # # Drop unnecessary columns that are not needed for recommendation
    # df = df.drop(columns=["track_album_id", "track_album_name", "track_album_release_date", 
    #                       "playlist_name", "playlist_id"])
    # Update the Column name
    df.rename(columns={'Music.1': 'Music'}, inplace=True)
    
    # Remove duplicate anime based on track_id and reset index again
    df = df.drop_duplicates(subset=["name"]).reset_index(drop=True)

    # Step 2: Define the columns for metadata and features
    meta_cols = ["anime_id","name"]  # Metadata columns to keep anime info
    feature_cols = ['episodes', 'rating', 'members', 'Action', 'Adventure', 'Cars',
       'Comedy', 'Dementia', 'Demons', 'Drama', 'Fantasy', 'Game',
       'Historical', 'Horror', 'Josei', 'Kids', 'Magic', 'MartialArts',
       'Mecha', 'Military', 'Music', 'Mystery', 'Parody', 'Police',
       'Psychological', 'Romance', 'Samurai', 'School', 'Sci-Fi', 'Seinen',
       'Shoujo', 'ShoujoAi', 'Shounen', 'ShounenAi', 'SliceofLife', 'Space',
       'Sports', 'SuperPower', 'Supernatural', 'Thriller', 'Vampire', 'Movie',
       'Music', 'ONA', 'OVA', 'Special', 'TV']

    # Step 3: Define preprocessing steps for the data

    # Preprocessing for numeric features: fill missing values with the mean and standardize the values
    numeric_features = []
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),  # Fill missing values with the mean
        ('scaler', StandardScaler())])  # Standardize features

    # Preprocessing for binary features (if any): fill missing values with most frequent and label encode
    binary_features = []
    binary_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent', missing_values=pd.NA)),  # Fill missing values
        ('label', OrdinalEncoder())])  # Convert binary features to ordinal values

    # Preprocessing for categorical features: fill missing values and apply one-hot encoding
    categorical_features = []
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent', missing_values=pd.NA)),  # Fill missing values
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])  # Apply one-hot encoding

    # Combine all preprocessing steps into a single preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),  # Numeric features preprocessing
            ('binary', binary_transformer, binary_features),  # Binary features preprocessing (if any)
            ('cat', categorical_transformer, categorical_features)])  # Categorical features preprocessing

    # Step 4: Get the target track's ID based on track name and artist
    # Select the most popular track by track_name and track_artist combination
    anime_id = df.loc[(df.name == anime_name)] \
                .sort_values(by="rating", ascending=False).anime_id.values[0]
    # anime_id = 22319
    # Step 5: Prepare the feature matrix for the Nearest Neighbors model
    X = df.loc[:, feature_cols]  # Select the feature columns

    # Apply preprocessing to the feature matrix
    # preprocessor.fit(X)  # Fit the preprocessor to the feature matrix
    # X_preprocessed = preprocessor.transform(X)  # Transform the feature matrix

    # Step 6: Initialize the Nearest Neighbors model
    # # define the number of nearest neighbors to consider and use cosine similarity as the metric
    k = anime_length
    model1 = NearestNeighbors(n_neighbors=k, metric="euclidean")

    # Fit the Nearest Neighbors model to the preprocessed data
    # model1.fit(X_preprocessed)
    model1.fit(X)
    # Step 7: Extract features of the anime
    anime_features = df.loc[df.anime_id == anime_id, feature_cols]  # Get the feature vector for the target anime
    # anime_features_preprocessed = preprocessor.transform(anime_features)  # Preprocess the target track features

    # Step 8: Find the nearest neighbors (anime most similar to the target selection)
    # distances, indices = model1.kneighbors(anime_features_preprocessed)  # Get distances and indices of neighbors
    distances, indices = model1.kneighbors(anime_features)  # Get distances and indices of neighbors

    # Step 9: Retrieve the metadata of the recommended tracks
    animes = df.iloc[indices[0]]  # Select anime corresponding to the nearest neighbors
    animes["distance"] = distances[0]  # Add the distance of each neighbor as a new column

    # Step 10: Filter the columns for the final output
    cols = animes.columns # you can explicitly choose to return specific columns here
    animes = animes.loc[:, cols]  # Keep the relevant columns
    animes = animes.sort_values(by="distance")  # Sort the tracks by their distance (most similar first)

    # Step 11: Return the recommended tracks as a list of dictionaries
    return animes.to_dict(orient="records")

In [ ]:
 # User input
anime_length = 10
anime_name = "Naruto"


response = make_recommendation_euc(anime_length, anime_name) # used for the flask app, returns JSON

# to test
pd.DataFrame(response)

In [8]:
def make_recommendation_man(anime_length, anime_name):
    # Step 1: Load the dataset
    df = pd.read_csv("anime_ml.csv")
    
    # Remove any rows with missing values and reset the index
    df = df.dropna(how="any").reset_index(drop=True)

    # # Extract the release year from the album release date and convert to integer
    # df["track_release_year"] = df.track_album_release_date.apply(lambda x: int(x[0:4]))

    # # Drop unnecessary columns that are not needed for recommendation
    # df = df.drop(columns=["track_album_id", "track_album_name", "track_album_release_date", 
    #                       "playlist_name", "playlist_id"])
    # Update the Column name
    df.rename(columns={'Music.1': 'Music'}, inplace=True)
    
    # Remove duplicate anime based on track_id and reset index again
    df = df.drop_duplicates(subset=["name"]).reset_index(drop=True)

    # Step 2: Define the columns for metadata and features
    meta_cols = ["anime_id","name"]  # Metadata columns to keep anime info
    feature_cols = ['episodes', 'rating', 'members', 'Action', 'Adventure', 'Cars',
       'Comedy', 'Dementia', 'Demons', 'Drama', 'Fantasy', 'Game',
       'Historical', 'Horror', 'Josei', 'Kids', 'Magic', 'MartialArts',
       'Mecha', 'Military', 'Music', 'Mystery', 'Parody', 'Police',
       'Psychological', 'Romance', 'Samurai', 'School', 'Sci-Fi', 'Seinen',
       'Shoujo', 'ShoujoAi', 'Shounen', 'ShounenAi', 'SliceofLife', 'Space',
       'Sports', 'SuperPower', 'Supernatural', 'Thriller', 'Vampire', 'Movie',
       'Music', 'ONA', 'OVA', 'Special', 'TV']

    # Step 3: Define preprocessing steps for the data

    # Preprocessing for numeric features: fill missing values with the mean and standardize the values
    numeric_features = []
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),  # Fill missing values with the mean
        ('scaler', StandardScaler())])  # Standardize features

    # Preprocessing for binary features (if any): fill missing values with most frequent and label encode
    binary_features = []
    binary_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent', missing_values=pd.NA)),  # Fill missing values
        ('label', OrdinalEncoder())])  # Convert binary features to ordinal values

    # Preprocessing for categorical features: fill missing values and apply one-hot encoding
    categorical_features = []
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent', missing_values=pd.NA)),  # Fill missing values
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])  # Apply one-hot encoding

    # Combine all preprocessing steps into a single preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),  # Numeric features preprocessing
            ('binary', binary_transformer, binary_features),  # Binary features preprocessing (if any)
            ('cat', categorical_transformer, categorical_features)])  # Categorical features preprocessing

    # Step 4: Get the target track's ID based on track name and artist
    # Select the most popular track by track_name and track_artist combination
    anime_id = df.loc[(df.name == anime_name)] \
                .sort_values(by="rating", ascending=False).anime_id.values[0]
    # anime_id = 22319
    # Step 5: Prepare the feature matrix for the Nearest Neighbors model
    X = df.loc[:, feature_cols]  # Select the feature columns

    # Apply preprocessing to the feature matrix
    # preprocessor.fit(X)  # Fit the preprocessor to the feature matrix
    # X_preprocessed = preprocessor.transform(X)  # Transform the feature matrix

    # Step 6: Initialize the Nearest Neighbors model
    # # define the number of nearest neighbors to consider and use cosine similarity as the metric
    k = anime_length
    model1 = NearestNeighbors(n_neighbors=k, metric="manhattan")

    # Fit the Nearest Neighbors model to the preprocessed data
    # model1.fit(X_preprocessed)
    model1.fit(X)
    # Step 7: Extract features of the anime
    anime_features = df.loc[df.anime_id == anime_id, feature_cols]  # Get the feature vector for the target anime
    # anime_features_preprocessed = preprocessor.transform(anime_features)  # Preprocess the target track features

    # Step 8: Find the nearest neighbors (anime most similar to the target selection)
    # distances, indices = model1.kneighbors(anime_features_preprocessed)  # Get distances and indices of neighbors
    distances, indices = model1.kneighbors(anime_features)  # Get distances and indices of neighbors

    # Step 9: Retrieve the metadata of the recommended tracks
    animes = df.iloc[indices[0]]  # Select anime corresponding to the nearest neighbors
    animes["distance"] = distances[0]  # Add the distance of each neighbor as a new column

    # Step 10: Filter the columns for the final output
    cols = animes.columns # you can explicitly choose to return specific columns here
    animes = animes.loc[:, cols]  # Keep the relevant columns
    animes = animes.sort_values(by="distance")  # Sort the tracks by their distance (most similar first)

    # Step 11: Return the recommended tracks as a list of dictionaries
    return animes.to_dict(orient="records")

In [ ]:
 # User input
anime_length = 10
anime_name = "Naruto"


response = make_recommendation_man(anime_length, anime_name) # used for the flask app, returns JSON

# to test
pd.DataFrame(response)

In [10]:
def make_recommendation(anime_length, anime_name, genres):
    # Step 1: Load the dataset
    df = pd.read_csv("anime_ml.csv")
    
    # Remove any rows with missing values and reset the index
    df = df.dropna(how="any").reset_index(drop=True)

    # Update the Column name
    df.rename(columns={'Music.1': 'Music'}, inplace=True)
    
    # Remove duplicate anime based on name
    df = df.drop_duplicates(subset=["name"]).reset_index(drop=True)

    # Define the columns for metadata and features
    meta_cols = ["anime_id", "name"]
    feature_cols = ['episodes', 'rating', 'members', 'Action', 'Adventure', 'Cars',
                    'Comedy', 'Dementia', 'Demons', 'Drama', 'Fantasy', 'Game',
                    'Historical', 'Horror', 'Josei', 'Kids', 'Magic', 'MartialArts',
                    'Mecha', 'Military', 'Music', 'Mystery', 'Parody', 'Police',
                    'Psychological', 'Romance', 'Samurai', 'School', 'Sci-Fi', 'Seinen',
                    'Shoujo', 'ShoujoAi', 'Shounen', 'ShounenAi', 'SliceofLife', 'Space',
                    'Sports', 'SuperPower', 'Supernatural', 'Thriller', 'Vampire', 'Movie',
                    'Music', 'ONA', 'OVA', 'Special', 'TV']

    # Preprocessing for numeric features
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ])
    
    # Categorical features
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

    # Combine all preprocessing steps
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, ['episodes', 'rating', 'members']),
            ('cat', categorical_transformer, feature_cols[3:])  # Adjust as necessary
        ]
    )

    # Step 4: Get the target anime's ID based on name
    anime_match = df.loc[df.name == anime_name]
    if anime_match.empty:
        raise ValueError(f"Anime '{anime_name}' not found.")
    
    anime_id = anime_match.sort_values(by="rating", ascending=False).anime_id.values[0]

    # Step 5: Filter for the specified genres
    genre_filter = df[genres].sum(axis=1) > 0  # Keep rows where at least one genre is present
    filtered_df = df[genre_filter]

    # If no anime matches the genres, raise an error
    if filtered_df.empty:
        raise ValueError(f"No anime found for genres: {', '.join(genres)}")

    # Prepare the feature matrix for the Nearest Neighbors model
    X = filtered_df[feature_cols]  # Select the feature columns
    X_preprocessed = preprocessor.fit_transform(X)  # Fit and transform the feature matrix

    # Step 6: Initialize and fit the Nearest Neighbors model
    k = anime_length
    model1 = NearestNeighbors(n_neighbors=k, metric="cosine")
    model1.fit(X_preprocessed)

    # Step 7: Extract features of the selected anime
    anime_features = df.loc[df.anime_id == anime_id, feature_cols]
    anime_features_preprocessed = preprocessor.transform(anime_features)

    # Step 8: Find the nearest neighbors
    distances, indices = model1.kneighbors(anime_features_preprocessed)

    # Step 9: Retrieve the metadata of the recommended animes
    animes = filtered_df.iloc[indices[0]]
    animes["distance"] = distances[0]  # Add distances to the DataFrame

    # Step 10: Filter the columns for the final output
    animes = animes.sort_values(by="distance")  # Sort by distance

    # Step 11: Return the recommended animes as a list of dictionaries
    return animes.to_dict(orient="records")

In [11]:
def make_recommendation(anime_length, anime_name, genres):
    # Load the dataset
    df = pd.read_csv("anime_ml.csv")

    # Remove any rows with missing values and reset the index
    df = df.dropna(how="any").reset_index(drop=True)

    # Update the column name if necessary
    df.rename(columns={'Music.1': 'Music'}, inplace=True)

    # Remove duplicate anime based on name
    df = df.drop_duplicates(subset=["name"]).reset_index(drop=True)

    # Define the columns for metadata and features
    meta_cols = ["anime_id", "name"]
    feature_cols = ['episodes', 'rating', 'members'] + genres  # Include boolean genre columns directly

    # Preprocessing for numeric features
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ])
    
    # Combine all preprocessing steps
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, ['episodes', 'rating', 'members'])
        ]
    )

    # Get the target anime's ID based on name
    anime_match = df.loc[df.name == anime_name]
    if anime_match.empty:
        raise ValueError(f"Anime '{anime_name}' not found.")
    
    anime_id = anime_match.sort_values(by="rating", ascending=False).anime_id.values[0]

    # Filter for the specified genres
    genre_filter = df[genres].sum(axis=1) > 0  # Keep rows where at least one genre is present
    filtered_df = df[genre_filter]

    # If no anime matches the genres, raise an error
    if filtered_df.empty:
        raise ValueError(f"No anime found for genres: {', '.join(genres)}")

    # Prepare the feature matrix for the Nearest Neighbors model
    X = filtered_df[feature_cols]  # Select the feature columns
    X_preprocessed = preprocessor.fit_transform(X)  # Fit and transform the feature matrix

    # Initialize and fit the Nearest Neighbors model
    k = anime_length
    model1 = NearestNeighbors(n_neighbors=k, metric="cosine")
    model1.fit(X_preprocessed)

    # Extract features of the selected anime
    anime_features = df.loc[df.anime_id == anime_id, feature_cols]
    anime_features_preprocessed = preprocessor.transform(anime_features)

    # Find the nearest neighbors
    distances, indices = model1.kneighbors(anime_features_preprocessed)

    # Retrieve the metadata of the recommended animes
    animes = filtered_df.iloc[indices[0]]
    animes["distance"] = distances[0]  # Add distances to the DataFrame

    # Filter the columns for the final output
    animes = animes.sort_values(by="distance")  # Sort by distance

    # Return the recommended animes as a list of dictionaries
    return animes.to_dict(orient="records")


In [12]:
anime_length = 10
anime_name = "Naruto"
genres = ["MartialArts"]  # Specify the genres you want to filter by

response = make_recommendation(anime_length, anime_name, genres)

In [ ]:
pd.DataFrame(response)

In [14]:
def make_recommendation(anime_length, anime_name, genres):
    # Load the dataset
    df = pd.read_csv("anime_ml.csv")

    # Remove any rows with missing values and reset the index
    df = df.dropna(how="any").reset_index(drop=True)

    # Update the column name if necessary
    df.rename(columns={'Music.1': 'Music'}, inplace=True)

    # Remove duplicate anime based on name
    df = df.drop_duplicates(subset=["name"]).reset_index(drop=True)

    # Define the columns for metadata and features
    meta_cols = ["anime_id", "name"]
    feature_cols = ['episodes', 'rating', 'members'] + genres  # Include boolean genre columns directly

    # Preprocessing for numeric features
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ])
    
    # Combine all preprocessing steps
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, ['episodes', 'rating', 'members'])
        ]
    )

    # Get the target anime's ID based on name
    anime_match = df.loc[df.name == anime_name]
    if anime_match.empty:
        raise ValueError(f"Anime '{anime_name}' not found.")
    
    anime_id = anime_match.sort_values(by="rating", ascending=False).anime_id.values[0]

    # Filter for the specified genres
    genre_filter = df[genres].sum(axis=1) > 0  # Keep rows where at least one genre is present
    filtered_df = df[genre_filter]

    # If no anime matches the genres, raise an error
    if filtered_df.empty:
        raise ValueError(f"No anime found for genres: {', '.join(genres)}")

    # Prepare the feature matrix for the Nearest Neighbors model
    X = filtered_df[feature_cols]  # Select the feature columns
    X_preprocessed = preprocessor.fit_transform(X)  # Fit and transform the feature matrix

    # Initialize and fit the Nearest Neighbors model
    k = anime_length
    model1 = NearestNeighbors(n_neighbors=k, metric="euclidean")
    model1.fit(X_preprocessed)

    # Extract features of the selected anime
    anime_features = df.loc[df.anime_id == anime_id, feature_cols]
    anime_features_preprocessed = preprocessor.transform(anime_features)

    # Find the nearest neighbors
    distances, indices = model1.kneighbors(anime_features_preprocessed)

    # Retrieve the metadata of the recommended animes
    animes = filtered_df.iloc[indices[0]]
    animes["distance"] = distances[0]  # Add distances to the DataFrame

    # Filter the columns for the final output
    animes = animes.sort_values(by="distance")  # Sort by distance

    # Return the recommended animes as a list of dictionaries
    return animes.to_dict(orient="records")

In [15]:
# User input
anime_length = 10
anime_name = "Naruto"
genres = ["MartialArts"]  # Specify the genres you want to filter by

response = make_recommendation(anime_length, anime_name, genres)

In [ ]:
# To test
pd.DataFrame(response)

In [4]:
def make_recommendation(anime_length, anime_name, genres):
    # Load the dataset
    df = pd.read_csv("anime_ml.csv")

    # Remove any rows with missing values and reset the index
    df = df.dropna(how="any").reset_index(drop=True)

    # Update the column name if necessary
    df.rename(columns={'Music.1': 'Music'}, inplace=True)

    # Remove duplicate anime based on name
    df = df.drop_duplicates(subset=["name"]).reset_index(drop=True)

    # Define the columns for metadata and features
    meta_cols = ["anime_id", "name"]
    feature_cols = ['episodes', 'rating', 'members'] + genres  # Include boolean genre columns directly

    # Preprocessing for numeric features
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ])
    
    # Combine all preprocessing steps
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, ['episodes', 'rating', 'members'])
        ]
    )

    # Get the target anime's ID based on name
    anime_match = df.loc[df.name == anime_name]
    if anime_match.empty:
        raise ValueError(f"Anime '{anime_name}' not found.")
    
    anime_id = anime_match.sort_values(by="rating", ascending=False).anime_id.values[0]

    # Filter for the specified genres
    genre_filter = df[genres].sum(axis=1) > 0  # Keep rows where at least one genre is present
    filtered_df = df[genre_filter]

    # If no anime matches the genres, raise an error
    if filtered_df.empty:
        raise ValueError(f"No anime found for genres: {', '.join(genres)}")

    # Prepare the feature matrix for the Nearest Neighbors model
    X = filtered_df[feature_cols]  # Select the feature columns
    X_preprocessed = preprocessor.fit_transform(X)  # Fit and transform the feature matrix

    # Initialize and fit the Nearest Neighbors model
    k = anime_length
    model1 = NearestNeighbors(n_neighbors=k, metric="manhattan")
    model1.fit(X_preprocessed)

    # Extract features of the selected anime
    anime_features = df.loc[df.anime_id == anime_id, feature_cols]
    anime_features_preprocessed = preprocessor.transform(anime_features)

    # Find the nearest neighbors
    distances, indices = model1.kneighbors(anime_features_preprocessed)

    # Retrieve the metadata of the recommended animes
    animes = filtered_df.iloc[indices[0]]
    animes["distance"] = distances[0]  # Add distances to the DataFrame

    # Filter the columns for the final output
    animes = animes.sort_values(by="distance")  # Sort by distance

    # Return the recommended animes as a list of dictionaries
    return animes.to_dict(orient="records")

In [5]:
anime_length = 10
anime_name = "Naruto"
genres = ["Action", "MartialArts", "Comedy"]  # Specify the genres you want to filter by

response = make_recommendation(anime_length, anime_name, genres) 

In [ ]:
# To test
pd.DataFrame(response)

Recommender without Name

In [ ]:
def make_recommendation(anime_length, anime_name=None, genres=None):
    # Load the dataset
    df = pd.read_csv("anime_ml.csv")

    # Remove any rows with missing values and reset the index
    df = df.dropna(how="any").reset_index(drop=True)

    # Update the column name if necessary
    df.rename(columns={'Music.1': 'Music'}, inplace=True)

    # Remove duplicate anime based on name
    df = df.drop_duplicates(subset=["name"]).reset_index(drop=True)

    # Define the columns for metadata and features
    meta_cols = ["anime_id", "name"]
    feature_cols = ['episodes', 'rating', 'members'] + genres if genres else []  # Include genres if provided

    # Preprocessing for numeric features
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ])
    
    # Combine all preprocessing steps
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, ['episodes', 'rating', 'members'])
        ]
    )

    # Determine which anime to base recommendations on
    if anime_name:
        anime_match = df.loc[df.name == anime_name]
        if anime_match.empty:
            raise ValueError(f"Anime '{anime_name}' not found.")
        anime_id = anime_match.sort_values(by="rating", ascending=False).anime_id.values[0]
    else:
        # Randomly select an anime if no name is provided
        anime_id = random.choice(df['anime_id'].values)

    # Filter for the specified genres
    if genres:
        genre_filter = df[genres].sum(axis=1) > 0  # Keep rows where at least one genre is present
        filtered_df = df[genre_filter]
    else:
        filtered_df = df

    # If no anime matches the genres, raise an error
    if filtered_df.empty:
        raise ValueError(f"No anime found for genres: {', '.join(genres)}")

    # Prepare the feature matrix for the Nearest Neighbors model
    X = filtered_df[feature_cols]  # Select the feature columns
    X_preprocessed = preprocessor.fit_transform(X)  # Fit and transform the feature matrix

    # Initialize and fit the Nearest Neighbors model
    k = anime_length
    model1 = NearestNeighbors(n_neighbors=k, metric="manhattan")
    model1.fit(X_preprocessed)

    # Extract features of the selected anime
    anime_features = df.loc[df.anime_id == anime_id, feature_cols]
    anime_features_preprocessed = preprocessor.transform(anime_features)

    # Find the nearest neighbors
    distances, indices = model1.kneighbors(anime_features_preprocessed)

    # Retrieve the metadata of the recommended animes
    animes = filtered_df.iloc[indices[0]]
    animes["distance"] = distances[0]  # Add distances to the DataFrame

    # Filter the columns for the final output
    animes = animes.sort_values(by="distance")  # Sort by distance

    # Return the recommended animes as a list of dictionaries
    return animes.to_dict(orient="records")

# Example usage
anime_length = 10
genres = ["Comedy"]  # Specify the genres you want to filter by

response = make_recommendation(anime_length, genres=genres)

# To test
pd.DataFrame(response)

Recommender without Name, and Type Filter

In [ ]:
def make_recommendation(anime_length, anime_name=None, genres=None, type_filter=None):
    # Load the dataset
    df = pd.read_csv("anime_ml.csv")

    # Remove any rows with missing values and reset the index
    df = df.dropna(how="any").reset_index(drop=True)

    # Update the column name if necessary
    df.rename(columns={'Music.1': 'Music'}, inplace=True)

    # Remove duplicate anime based on name
    df = df.drop_duplicates(subset=["name"]).reset_index(drop=True)

    # Define the columns for metadata and features
    meta_cols = ["anime_id", "name"]
    feature_cols = ['episodes', 'rating', 'members'] + genres if genres else []  # Include genres if provided

    # Preprocessing for numeric features
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ])
    
    # Combine all preprocessing steps
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, ['episodes', 'rating', 'members'])
        ]
    )

    # Determine which anime to base recommendations on
    if anime_name:
        anime_match = df.loc[df.name == anime_name]
        if anime_match.empty:
            raise ValueError(f"Anime '{anime_name}' not found.")
        anime_id = anime_match.sort_values(by="rating", ascending=False).anime_id.values[0]
    else:
        # Randomly select an anime if no name is provided
        anime_id = random.choice(df['anime_id'].values)

    # Apply genre filter if specified
    if genres:
        genre_filter = df[genres].sum(axis=1) > 0  # Keep rows where at least one genre is present
        filtered_df = df[genre_filter]
    else:
        filtered_df = df

    # Apply type filter if specified
    if type_filter:
        type_filter_condition = filtered_df['type'].isin(type_filter)  # Keep rows matching the specified types
        filtered_df = filtered_df[type_filter_condition]

    # If no anime matches the filters, raise an error
    if filtered_df.empty:
        raise ValueError("No anime found for the specified filters.")

    # Prepare the feature matrix for the Nearest Neighbors model
    X = filtered_df[feature_cols]  # Select the feature columns
    X_preprocessed = preprocessor.fit_transform(X)  # Fit and transform the feature matrix

    # Initialize and fit the Nearest Neighbors model
    k = anime_length
    model1 = NearestNeighbors(n_neighbors=k, metric="manhattan")
    model1.fit(X_preprocessed)

    # Extract features of the selected anime
    anime_features = df.loc[df.anime_id == anime_id, feature_cols]
    anime_features_preprocessed = preprocessor.transform(anime_features)

    # Find the nearest neighbors
    distances, indices = model1.kneighbors(anime_features_preprocessed)

    # Retrieve the metadata of the recommended animes
    animes = filtered_df.iloc[indices[0]]
    animes["distance"] = distances[0]  # Add distances to the DataFrame

    # Filter the columns for the final output
    animes = animes.sort_values(by="distance")  # Sort by distance

    # Return the recommended animes as a list of dictionaries
    return animes.to_dict(orient="records")

# Example usage
anime_length = 10
genres = ["Action", "MartialArts", "Comedy"]  # Specify the genres you want to filter by
type_filter = ["TV", "Movie"]  # Specify the types to filter by

response = make_recommendation(anime_length, genres=genres, type_filter=type_filter)

# To test
pd.DataFrame(response)

Forgot that they types are Booleans

In [ ]:
import random

def make_recommendation(anime_length, anime_name=None, genres=None, type_filter=None):
    # Load the dataset
    df = pd.read_csv("anime_ml.csv")

    # Remove any rows with missing values and reset the index
    df = df.dropna(how="any").reset_index(drop=True)

    # Update the column name if necessary
    df.rename(columns={'Music.1': 'Music'}, inplace=True)

    # Remove duplicate anime based on name
    df = df.drop_duplicates(subset=["name"]).reset_index(drop=True)

    # Define the columns for metadata and features
    meta_cols = ["anime_id", "name"]
    feature_cols = ['episodes', 'rating', 'members'] + genres if genres else []  # Include genres if provided

    # Preprocessing for numeric features
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ])
    
    # Combine all preprocessing steps
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, ['episodes', 'rating', 'members'])
        ]
    )

    # Determine which anime to base recommendations on
    if anime_name:
        anime_match = df.loc[df.name == anime_name]
        if anime_match.empty:
            raise ValueError(f"Anime '{anime_name}' not found.")
        anime_id = anime_match.sort_values(by="rating", ascending=False).anime_id.values[0]
    else:
        # Randomly select an anime if no name is provided
        anime_id = random.choice(df['anime_id'].values)

    # Apply genre filter if specified
    filtered_df = df
    if genres:
        genre_filter = filtered_df[genres].sum(axis=1) > 0  # Keep rows where at least one genre is present
        filtered_df = filtered_df[genre_filter]

    # Apply type filter if specified
    if type_filter:
        type_conditions = [filtered_df[type] == True for type in type_filter if type in filtered_df.columns]
        if type_conditions:
            combined_condition = np.logical_or.reduce(type_conditions)
            filtered_df = filtered_df[combined_condition]

    # If no anime matches the filters, raise an error
    if filtered_df.empty:
        raise ValueError("No anime found for the specified filters.")

    # Prepare the feature matrix for the Nearest Neighbors model
    X = filtered_df[feature_cols]  # Select the feature columns
    X_preprocessed = preprocessor.fit_transform(X)  # Fit and transform the feature matrix

    # Initialize and fit the Nearest Neighbors model
    k = anime_length
    model1 = NearestNeighbors(n_neighbors=k, metric="manhattan")
    model1.fit(X_preprocessed)

    # Extract features of the selected anime
    anime_features = df.loc[df.anime_id == anime_id, feature_cols]
    anime_features_preprocessed = preprocessor.transform(anime_features)

    # Find the nearest neighbors
    distances, indices = model1.kneighbors(anime_features_preprocessed)

    # Retrieve the metadata of the recommended animes
    animes = filtered_df.iloc[indices[0]]
    animes["distance"] = distances[0]  # Add distances to the DataFrame

    # Filter the columns for the final output
    animes = animes.sort_values(by="distance")  # Sort by distance

    # Return the recommended animes as a list of dictionaries
    return animes.to_dict(orient="records")

# Example usage
anime_length = 10
genres = ["Comedy"]  # Specify the genres you want to filter by
type_filter = ["Movie"]  # Specify the types to filter by as a list of column names

response = make_recommendation(anime_length, genres=genres, type_filter=type_filter)

# To test
pd.DataFrame(response)


In [ ]:
def make_recommendation(anime_length, anime_name=None, genres=None, type_filter=None, rating_min=None):
    # Load the dataset
    df = pd.read_csv("anime_ml.csv")

    # Remove any rows with missing values and reset the index
    df = df.dropna(how="any").reset_index(drop=True)

    # Update the column name if necessary
    df.rename(columns={'Music.1': 'Music'}, inplace=True)

    # Remove duplicate anime based on name
    df = df.drop_duplicates(subset=["name"]).reset_index(drop=True)

    # Define the columns for metadata and features
    meta_cols = ["anime_id", "name"]
    feature_cols = ['episodes', 'rating', 'members'] + (genres if genres else [])  # Include genres if provided

    # Preprocessing for numeric features
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ])
    
    # Combine all preprocessing steps
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, ['episodes', 'rating', 'members'])
        ]
    )

    # Determine which anime to base recommendations on
    if anime_name:
        anime_match = df.loc[df.name == anime_name]
        if anime_match.empty:
            raise ValueError(f"Anime '{anime_name}' not found.")
        anime_id = anime_match.sort_values(by="rating", ascending=False).anime_id.values[0]
    else:
        # Randomly select an anime if no name is provided
        anime_id = random.choice(df['anime_id'].values)

    # Apply genre filter if specified
    filtered_df = df
    if genres:
        genre_filter = filtered_df[genres].sum(axis=1) > 0  # Keep rows where at least one genre is present
        filtered_df = filtered_df[genre_filter]

    # Apply type filter if specified
    if type_filter:
        type_conditions = [filtered_df[type] == True for type in type_filter if type in filtered_df.columns]
        if type_conditions:
            combined_condition = np.logical_or.reduce(type_conditions)
            filtered_df = filtered_df[combined_condition]

    # Apply minimum rating filter if specified
    if rating_min is not None:
        filtered_df = filtered_df[filtered_df['rating'] >= rating_min]

    # If no anime matches the filters, raise an error
    if filtered_df.empty:
        raise ValueError("No anime found for the specified filters.")

    # Prepare the feature matrix for the Nearest Neighbors model
    X = filtered_df[feature_cols]  # Select the feature columns
    X_preprocessed = preprocessor.fit_transform(X)  # Fit and transform the feature matrix

    # Initialize and fit the Nearest Neighbors model
    k = anime_length
    model1 = NearestNeighbors(n_neighbors=k, metric="manhattan")
    model1.fit(X_preprocessed)

    # Extract features of the selected anime
    anime_features = df.loc[df.anime_id == anime_id, feature_cols]
    anime_features_preprocessed = preprocessor.transform(anime_features)

    # Find the nearest neighbors
    distances, indices = model1.kneighbors(anime_features_preprocessed)

    # Retrieve the metadata of the recommended animes
    animes = filtered_df.iloc[indices[0]]
    animes["distance"] = distances[0]  # Add distances to the DataFrame

    # Filter the columns for the final output
    animes = animes.sort_values(by="distance")  # Sort by distance

    # Return the recommended animes as a list of dictionaries
    return animes.to_dict(orient="records")

# Example usage
anime_length = 5
anime_name = "Naruto"
genres = ["Comedy"]  # Specify the genres you want to filter by
type_filter = ["TV"]  # Specify the types to filter by
rating_min = None  # Minimum rating (can be None if no minimum is needed)

response = make_recommendation(anime_length, genres=genres, type_filter=type_filter, rating_min=rating_min)

# To test
pd.DataFrame(response)

In [4]:
def make_recommendation(anime_length, anime_name=None, genres=None, type_filter=None, rating_min=None, episodes_min=None):
    # Load the dataset
    df = pd.read_csv("anime_ml.csv")

    # Remove any rows with missing values and reset the index
    df = df.dropna(how="any").reset_index(drop=True)

    # Update the column name if necessary
    df.rename(columns={'Music.1': 'Music'}, inplace=True)

    # Remove duplicate anime based on name
    df = df.drop_duplicates(subset=["name"]).reset_index(drop=True)

    # Define the columns for metadata and features
    meta_cols = ["anime_id", "name"]
    feature_cols = ['episodes', 'rating', 'members'] + (genres if genres else [])  # Include genres if provided

    # Preprocessing for numeric features
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ])
    
    # Combine all preprocessing steps
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, ['episodes', 'rating', 'members'])
        ]
    )

    # Determine which anime to base recommendations on
    if anime_name:
        anime_match = df.loc[df.name == anime_name]
        if anime_match.empty:
            raise ValueError(f"Anime '{anime_name}' not found.")
        anime_id = anime_match.sort_values(by="rating", ascending=False).anime_id.values[0]
    else:
        # Randomly select an anime if no name is provided
        anime_id = random.choice(df['anime_id'].values)

    # Apply genre filter if specified
    filtered_df = df
    if genres:
        genre_filter = filtered_df[genres].sum(axis=1) > 0  # Keep rows where at least one genre is present
        filtered_df = filtered_df[genre_filter]

    # Apply type filter if specified
    if type_filter:
        type_conditions = [filtered_df[type] == True for type in type_filter if type in filtered_df.columns]
        if type_conditions:
            combined_condition = np.logical_or.reduce(type_conditions)
            filtered_df = filtered_df[combined_condition]

    # Apply minimum rating filter if specified
    if rating_min is not None:
        filtered_df = filtered_df[filtered_df['rating'] >= rating_min]

    # Apply minimum episodes filter if specified
    if episodes_min is not None:
        filtered_df = filtered_df[filtered_df['episodes'] >= episodes_min]

    # If no anime matches the filters, raise an error
    if filtered_df.empty:
        raise ValueError("No anime found for the specified filters.")

    # Prepare the feature matrix for the Nearest Neighbors model
    X = filtered_df[feature_cols]  # Select the feature columns
    X_preprocessed = preprocessor.fit_transform(X)  # Fit and transform the feature matrix

    # Initialize and fit the Nearest Neighbors model
    k = anime_length
    model1 = NearestNeighbors(n_neighbors=k, metric="manhattan")
    model1.fit(X_preprocessed)

    # Extract features of the selected anime
    anime_features = df.loc[df.anime_id == anime_id, feature_cols]
    anime_features_preprocessed = preprocessor.transform(anime_features)

    # Find the nearest neighbors
    distances, indices = model1.kneighbors(anime_features_preprocessed)

    # Retrieve the metadata of the recommended animes
    animes = filtered_df.iloc[indices[0]]
    animes["distance"] = distances[0]  # Add distances to the DataFrame

    # Filter the columns for the final output
    animes = animes.sort_values(by="distance")  # Sort by distance

    # Return the recommended animes as a list of dictionaries
    return animes.to_dict(orient="records")

# Example usage
anime_length = 10
anime_name = "Naruto"
genres = ["Comedy"]  # Specify the genres you want to filter by 
type_filter = ["TV"]  # Specify the types to filter by
rating_min = None # Minimum rating (can be None if no minimum is needed)
episodes_min = 1  # Minimum episodes (can be None if no minimum is needed)

response = make_recommendation(anime_length, genres=genres, type_filter=type_filter, rating_min=rating_min, episodes_min=episodes_min)

# To test
pd.DataFrame(response)

,anime_id,name,episodes,rating,members,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Fantasy,Game,Historical,Horror,Josei,Kids,Magic,MartialArts,Mecha,Military,Music,Mystery,Parody,Police,Psychological,Romance,Samurai,School,Sci-Fi,Seinen,Shoujo,ShoujoAi,Shounen,ShounenAi,SliceofLife,Space,Sports,SuperPower,Supernatural,Thriller,Vampire,Movie,ONA,OVA,Special,TV,distance
0,636,Juubee-chan 2: Siberia Yagyuu no Gyakushuu,13,6.99,6733,0,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0.029627
1,750,Binchou-tan,12,7.00,6960,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0.031970
2,5967,Kinnikuman II Sei: Ultimate Muscle,13,6.99,2435,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0.043973
3,1578,Koisuru Tenshi Angelique: Kokoro no Mezameru Toki,13,7.03,6786,0,1,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.053230
4,1837,Dokkoida,12,6.98,3462,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0.054524
5,630,Mahoutsukai Tai!,13,6.95,5328,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.059036
6,5968,Kinnikuman II Sei: Ultimate Muscle 2,13,7.03,2019,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0.071851
7,3519,Garakuta-doori no Stain,13,7.02,641,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.076592
8,580,Kogepan,10,7.04,4436,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.088346
9,1509,Mamotte! Lollipop,13,6.99,11747,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.088683


In [5]:
def make_recommendation(anime_length, anime_name=None, genres=None, type_filter=None, rating_min=None, episodes_min=None):
    # Load the dataset
    df = pd.read_csv("anime_ml.csv")

    # Remove any rows with missing values and reset the index
    df = df.dropna(how="any").reset_index(drop=True)

    # Update the column name if necessary
    df.rename(columns={'Music.1': 'Music'}, inplace=True)

    # Remove duplicate anime based on name
    df = df.drop_duplicates(subset=["name"]).reset_index(drop=True)

    # Define the columns for metadata and features
    meta_cols = ["anime_id", "name"]
    feature_cols = ['episodes', 'rating', 'members'] + (genres if genres else [])  # Include genres if provided

    # Preprocessing for numeric features
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ])
    
    # Combine all preprocessing steps
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, ['episodes', 'rating', 'members'])
        ]
    )

    # Determine which anime to base recommendations on
    if anime_name:
        anime_match = df.loc[df.name == anime_name]
        if anime_match.empty:
            raise ValueError(f"Anime '{anime_name}' not found.")
        anime_id = anime_match.sort_values(by="rating", ascending=False).anime_id.values[0]
    else:
        # Randomly select an anime if no name is provided
        anime_id = random.choice(df['anime_id'].values)

    # Apply genre filter if specified
    filtered_df = df
    if genres is not None and genres:
        genre_filter = filtered_df[genres].sum(axis=1) > 0  # Keep rows where at least one genre is present
        filtered_df = filtered_df[genre_filter]

    # Apply type filter if specified
    if type_filter is not None and type_filter:
        type_conditions = [filtered_df[type] == True for type in type_filter if type in filtered_df.columns]
        if type_conditions:
            combined_condition = np.logical_or.reduce(type_conditions)
            filtered_df = filtered_df[combined_condition]

    # Apply minimum rating filter if specified
    if rating_min is not None:
        filtered_df = filtered_df[filtered_df['rating'] >= rating_min]

    # Apply minimum episodes filter if specified
    if episodes_min is not None:
        filtered_df = filtered_df[filtered_df['episodes'] >= episodes_min]

    # If no anime matches the filters, raise an error
    if filtered_df.empty:
        raise ValueError("No anime found for the specified filters.")

    # Prepare the feature matrix for the Nearest Neighbors model
    X = filtered_df[feature_cols]  # Select the feature columns
    X_preprocessed = preprocessor.fit_transform(X)  # Fit and transform the feature matrix

    # Initialize and fit the Nearest Neighbors model
    k = anime_length
    model1 = NearestNeighbors(n_neighbors=k, metric="manhattan")
    model1.fit(X_preprocessed)

    # Extract features of the selected anime
    anime_features = df.loc[df.anime_id == anime_id, feature_cols]
    anime_features_preprocessed = preprocessor.transform(anime_features)

    # Find the nearest neighbors
    distances, indices = model1.kneighbors(anime_features_preprocessed)

    # Retrieve the metadata of the recommended animes
    animes = filtered_df.iloc[indices[0]]
    animes["distance"] = distances[0]  # Add distances to the DataFrame

    # Filter the columns for the final output
    animes = animes.sort_values(by="distance")  # Sort by distance

    # Return the recommended animes as a list of dictionaries
    return animes.to_dict(orient="records")

# Example usage
anime_length = 10
anime_name = None  # Can be None if no specific anime is needed
genres = ["Comedy"]  # Specify the genres you want to filter by (can be None)
type_filter = None  # Specify the types to filter by (can be None)
rating_min = None  # Minimum rating (can be None if no minimum is needed)
episodes_min = 1  # Minimum episodes (can be None if no minimum is needed)

response = make_recommendation(anime_length, anime_name=anime_name, genres=genres, type_filter=type_filter, rating_min=rating_min, episodes_min=episodes_min)

# To test
pd.DataFrame(response)

,anime_id,name,episodes,rating,members,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Fantasy,Game,Historical,Horror,Josei,Kids,Magic,MartialArts,Mecha,Military,Music,Mystery,Parody,Police,Psychological,Romance,Samurai,School,Sci-Fi,Seinen,Shoujo,ShoujoAi,Shounen,ShounenAi,SliceofLife,Space,Sports,SuperPower,Supernatural,Thriller,Vampire,Movie,ONA,OVA,Special,TV,distance
0,16389,Komachi to Dangorou,2,5.38,215,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.000000
1,17163,Peeping Life: The Perfect Evolution Specials,3,5.38,220,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0.017031
2,7449,Pacusi no Uta,1,5.38,225,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.017117
3,13087,Youkoso Uchuujin,1,5.38,190,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0.017375
4,9303,Noel no Fushigi na Bouken,1,5.38,133,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0.018355
5,22511,Kobutori,1,5.38,125,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0.018493
6,25061,Kizu darake no Tenshi-tachi,1,5.38,103,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.018871
7,21971,Sore Ike! Anpanman: Yuuki no Hana ga Hiraku Toki,1,5.38,75,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0.019352
8,21951,Sore Ike! Anpanman: Hiya Hiya Hiyariko to Bafu...,1,5.38,71,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0.019421
9,27493,Sore Ike! Anpanman: Yuuki to Honoo to Christmas,1,5.38,63,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.019559


In [9]:
with open('anime.pkl', 'wb') as file:
        pkl.dump(df, file)